<a href="https://colab.research.google.com/github/kaitlinblakeslee/DS2002F24/blob/main/DataProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Data processor should be able to fetch/download/retrieve local file or remote URL

import csv
import sqlite3
import json
import pandas as pd
import requests

# Ask the user for an input and an output data file and data type

input_file = input('Choose your input')
input_type = input('Choose your input type')
output_file = input('Choose a target ouput')
output_type = input('Choose your output type')







Choose your inputadvice_local_file.json
Choose your input typejson
Choose a target ouputadvice.sql
Choose your output typesqlite


In [ ]:
# Replace with your API URL - this is specifically for my JSON file which I got from the internet (a url)

import requests
import json

url = 'https://api.adviceslip.com/advice'

# Fetch the data
response = requests.get(url)
data = response.json()

# Write the data to a local file
with open("advice_local_file.json", "w") as advice_json_file:
    json.dump(data, advice_json_file, indent=4)

In [ ]:
# convert general format and data structure of the data source

# conversion function should do every single type of conversion I need

def convert_data(input_file, input_type, output_file, output_type):
  if input_type == 'csv' and output_type == 'json': #csv to json example
    with open(input_file, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = [row for row in csv_reader]
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file)
    print("CSV to JSON conversion successful!")
  elif input_type == 'json' and output_type == 'csv': #json to csv example
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)
    with open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=data[0].keys())
        csv_writer.writeheader()
        csv_writer.writerows(data)
    print("JSON to CSV conversion successful!")
  elif input_type == 'json' and output_type == 'sqlite': #json to sqlite example
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)
    with sqlite3.connect(output_file) as conn:
        df = pd.DataFrame(data)
        conn = sqlite3.connect(output_file)
        # need to define the table
        table = 'advice'
        df.to_sql(table, conn, if_exists='replace', index=False)
        conn.close()
    print("JSON to SQLite conversion successful!")
  elif input_type == 'sqlite' and output_type == 'csv': #sqlite to csv example
    with sqlite3.connect(input_file) as conn:
        table = 'advice' # define table again
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(output_file, index=False)
    print("SQLite to CSV conversion successful!")
  else:
    print("Invalid conversion") #handle errors - this is a way to deal with errors

convert_data(input_file, input_type, output_file, output_type)

JSON to SQLite conversion successful!


In [ ]:
# Modify the number of columns from the source to the destination

#Modify csv (source) dataset (WHR2023 - World Happiness Ratings) OR JSON data (depending on user's original input)

# csv deletion of columns
columns_to_delete = ['upperwhisker', 'lowerwhisker'] #specifically delete the upperwhisker and lowerwhisker columns from the happiness csv data

# json modification - did not add a column because nested within a dictionary (actually added a record and deleted a column)
new_columns = {
    "happy thoughts": "Golden Retriever puppies",
    "Rating": "Please leave a review and rate how helpful our advice was"
}

# Modify the number of columns function - handles inputs that are csvs or jsons

def modify_columns(input_file, input_type, output_file, output_type):
  if input_type == 'csv' and output_type == 'json':
    df = pd.read_csv(input_file)
    if columns_to_delete:
      df = df.drop(columns=columns_to_delete)
    df.to_json(output_file, orient='records')
    print("CSV to JSON conversion with column deletion successful!")
  elif input_type == 'json' and output_type == 'sqlite':
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)


# Check if 'slip' key exists and if it's a dictionary

    if 'slip' in data and isinstance(data['slip'], dict):
        # Apply column additions to the nested dictionary under 'slip'
        for key, value in new_columns.items():
            data['slip'][key] = value  # Modify this line to reflect correct path
        print("JSON to SQLite conversion with column addition successful!")
  else:
    print("Invalid conversion") # deal with errors

modify_columns(input_file, input_type, output_file, output_type)




JSON to SQLite conversion with column addition successful!


In [ ]:
# Store : the converted file should be written to a SQL database

#the json file that was converted to a sql is already written to a SQL database (advice.sql)

# however the csv file that was converted to a json has not yet been written to a SQL database

# write JSON file (WHR2023.json) to SQL database

# first load converted JSON file
with open("WHR2023.json", "r") as json_file:
    data = json.load(json_file)

# connect to SQLite
conn = sqlite3.connect('WHR.db')
cursor = conn.cursor()

# create table dynamically based on the JSON
columns = data[0].keys()  # Extract column names from the JSON keys
column_definitions = ", ".join([f"'{col}' TEXT" for col in columns])  # experienced issue with syntax so had to add 'back ticks' around column

# create table
cursor.execute(f"CREATE TABLE IF NOT EXISTS WHR_table ({column_definitions})")

# insert data from JSON into table
for item in data:
    values = [str(item[col]) for col in columns]
    placeholders = ", ".join(["?"] * len(columns))
    cursor.execute(f"INSERT INTO WHR_table ({', '.join([f'[{col}]' for col in columns])}) VALUES ({placeholders})", values) #also needed to enclose column names here

# commit changes and close connection
conn.commit()
conn.close()

print("Data from JSON file has been successfully written to a SQL database")





Data from JSON file has been successfully written to a SQL database


In [ ]:
# Data File ingestion summary : number of records and number of columns

# First - data file ingestion summary for csv
with open("WHR2023.csv", newline='') as csv_file:
    reader = csv.reader(csv_file)
    csv_data = list(reader)  # Convert to a list of rows

num_csv_records = len(csv_data)  # Number of records
num_csv_columns = len(csv_data[0])  # Number of columns

print(f"CSV Data Ingestion Summary: {num_csv_records} records, {num_csv_columns} columns.")

# Second - data file ingestion summary for json

with open("advice_local_file.json", "r") as json_file:
    data = json.load(json_file)

num_records = 1  # Number of records - this is because all of the data is nested in a dictionary ('slip')
num_columns = len(data['slip'].keys())  # Number of columns

print(f"JSON Data Ingestion Summary: {num_records} records, {num_columns} columns.")

CSV Data Ingestion Summary: 138 records, 19 columns.
JSON Data Ingestion Summary: 1 records, 2 columns.


In [ ]:
# Data File processing summary : number of records and number of columns

# First - data file processing summary for csv to json to sql

conn = sqlite3.connect('WHR.db')
cursor = conn.cursor()

# Count the number of records in the SQL table
cursor.execute('SELECT COUNT(*) FROM WHR_table')
num_records_sql = cursor.fetchone()[0]

# Count the number of columns in the SQL table
cursor.execute('PRAGMA table_info(WHR_table)')
num_columns_sql = len(cursor.fetchall())

print(f"Post-Processing Summary: {num_records_sql} records, {num_columns_sql} columns.")

# Close the connection
conn.close()


Post-Processing Summary: 274 records, 17 columns.


In [ ]:
# Second - data file processing summary for json to sql

conn = sqlite3.connect('advice.sql')
cursor = conn.cursor()

# Count the number of records in the SQL table
cursor.execute('SELECT COUNT(*) FROM advice')
num_records_sql = cursor.fetchone()[0]

# Count the number of columns in the SQL table
cursor.execute('PRAGMA table_info(advice)')
num_columns_sql = len(cursor.fetchall())

print(f"Post-Processing Summary: {num_records_sql} records, {num_columns_sql} columns.")

# Close the connection
conn.close()

Post-Processing Summary: 2 records, 1 columns.


One Page Reflection:

For me personally with minimal coding experience, this task was extremely daunting at first. I think I originally was confused at the task at hand and the logic for what exactly I had to accomplish. However, once I began creating the ETL processor, I started to get a better sense of the assignment, and it started to get easier. As a whole, I feel that I learned a lot through this project about different types of data and how to handle/convert them, which is a useful skill for any data science project.

The datasets I used were a ‘World Happiness Ratings’ csv file from Kaggle and an API url from the public github about ‘advice’. I initially thought that reading the data and getting the user inputs would be the easiest parts of the project, however I actually struggled with this quite a bit. I think starting any code is challenging because at this point I was still thinking through the steps I needed to take to create the processor. I named the input for my file ‘inputfile’, and I think this was interfering with the input function, because I originally kept getting an error. I changed the name to ‘input_file’, and this worked better. Additionally, even though we practiced API calls in class, I still don’t feel too comfortable yet working with API and JSON files, so converting the url to a local JSON file was also challenging for me. Going into the project, I thought that converting between file types would be the hardest part. However once I created the function and got started, it actually became quite repetitive. I think that creating a function to do this with ‘if’ statements made the most sense for this assignment. I also used ‘else’ to handle possible errors or if there is an unfamiliar data type. I forgot that I needed a table for the SQL data, so I did have to go back and define a table within the function. The modification for the csv file was fairly simple. For the JSON, on the other hand, I found myself very confused. This was because the JSON was made of a dictionary of dictionaries (and also some strings), so it was difficult to access the data directly. Instead, I used the ‘slip’ key to get inside the dictionary. I think spending more time familiarizing myself with the data in the beginning would’ve been beneficial in this case. Writing the original CSV to a SQL database was also a little challenging in that I thought I was going to have to create the table from scratch and add all the columns and values, which would’ve been extremely time consuming (and I did start to do this). However, then I realized I could just build the table dynamically from the JSON data, which was a lot quicker! Lastly, retrieving the data file ingestion and processing summary at the very end was probably the smoothest part of this project.

I think this data processing utility will be extremely useful for future data science projects. Often, the data is not presented in the format that is needed for a specific task or analytic model. So being able to convert relatively easily between data types using this processor is a great tool to have. While the datasets I used only had slight correlations to one another (world happiness ratings and therapeutic advice), I could see this tool being helpful when trying to compare similar datasets of different formats as well.
